### 数据读取

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
from datasets import Dataset, DatasetDict 
from collections import Counter
#from high_attention import kmer2seq, plot_random
import matplotlib.pyplot as plt

dataset_name="BERT_HERV_Multi_RUN0"
output_dir="/home/u20111010010/Project/DNA-Pretraining/Level1/003.Sequence_Visualization/Dataset_HERV"

In [ ]:
file="/home/u20111010010/Project/DNA-Pretraining/Level1/001.Genomics_dataset/Dataset_HERV/VCF_hprc-1000G/Train_Test/data_all_model_HERV-Classification_Need.fa"
df1=pd.read_csv(file,sep="\t",header=None).rename(columns = {0: "dset", 1: "multi",2:"binary", 3: "seq",4:"Type",5: "detail"})
df = df1.loc[:, ['dset', 'multi','seq']]
df = df[df['dset'] == 'test']

print("+++++++++++++++++++++++++++++Test sets")
print(df.shape)


### label间转换
id2label={"0":"Non-HERV_Coding","1":"HERV_Coding","2":"Non-HERV_Non-Coding","3":"HERV_Non-Coding"}
labels_raw=list(df['multi'])
labels = [id2label[str(i)] for i in labels_raw]

print(Counter(labels))

sequences=list(df['seq'])
#df["seq1"] = df["seq"].apply(kmer2seq)

In [ ]:
### 字体设置
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from matplotlib.font_manager import FontManager
FontManager()._findfont_cached.cache_clear()

#font_path = "/home/u20111010010/Project/DNA-Pretraining/Level1/Manuscript/Times_New_Roman/Times New Roman.ttf"

### 正常字体
font_path ="/home/u20111010010/Project/DNA-Pretraining/Level1/Manuscript/Times_New_Roman/Times_New_Roman.ttf"

### 粗体
#font_path ="/home/u20111010010/Project/DNA-Pretraining/Level1/Manuscript/Times_New_Roman/Times New Roman Bold.ttf"
#plt.rcParams['font.family'] = 'Times New Roman'
#plt.rcParams['font.serif'] = font_path

import matplotlib.font_manager as fm
fm.fontManager.addfont(font_path)
plt.rcParams['font.family'] = 'Times New Roman'

### 定义函数

In [ ]:
def contiguous_regions(condition, len_thres=5):
    """
    从布尔数组 "condition" 中查找连续的 True 区域。
    返回一个二维数组，其中第一列是区域的开始索引，第二列是结束索引。
    """
    # 找到 "condition" 中变化的索引
    d = np.diff(condition)
    (idx,) = d.nonzero()
    # 因为我们需要在 "condition" 变化之后开始，所以右移一位
    idx += 1
    if condition[0]:
        # 如果 condition 开始为 True，前面加一个 0
        idx = np.r_[0, idx]
    if condition[-1]:
        # 如果 condition 结束为 True，添加数组的长度
        idx = np.r_[idx, condition.size]
    # 将结果重塑为两列
    idx.shape = (-1, 2)
    # 剔除那些不满足长度阈值的区域
    idx = idx[np.argwhere((idx[:, 1] - idx[:, 0]) >= len_thres).flatten()]
    return idx

In [ ]:
def find_high_attention(score, min_len=5, **kwargs):
    """
    使用注意力得分数组作为输入，找到长度大于 min_len 的连续高注意力子区域的索引。
    """
    # 创建两个条件来定义"高注意力"
    cond1 = score > np.mean(score)
    cond2 = score > 10 * np.min(score)
    # 合并这两个条件
    cond = [cond1, cond2]
    cond = list(map(all, zip(*cond)))
    # 如果提供了自定义条件，则使用它们
    if "cond" in kwargs:
        cond = kwargs["cond"]
        if any(isinstance(x, list) for x in cond):
            cond = list(map(all, zip(*cond)))
    cond = np.asarray(cond)
    # 查找具有高注意力的连续区域
    motif_regions = contiguous_regions(cond, min_len)
    return motif_regions

In [ ]:
def token_range_to_dna_range(start_token_idx, end_token_idx, dna_length, kmer_size=6):
    actual_kmers = dna_length - kmer_size + 1  # 实际的 k-mer 数量
    # 如果 start_token_idx 超过了 actual_kmers，返回空
    if start_token_idx >= actual_kmers:
        return 0, 0
    # 如果 end_token_idx 超过了 actual_kmers，将其限制为 actual_kmers - 1
    if end_token_idx >= actual_kmers:
        end_token_idx = actual_kmers - 1
    # 判断 start_token 是否在前 300 个
    if start_token_idx < 300:
        start_dna_idx = start_token_idx
    else:
        # 为后212个tokens计算偏移量
        offset = start_token_idx - 300
        start_dna_idx = actual_kmers - (212 - offset)
    # 判断 end_token 是否在前 300 个
    if end_token_idx < 300:
        end_dna_idx = end_token_idx + kmer_size - 1
    else:
        # 为后212个tokens计算偏移量
        offset = end_token_idx - 300
        end_dna_idx = actual_kmers - (212 - offset) + kmer_size - 1
    return start_dna_idx, end_dna_idx

In [ ]:
def attention_seq(attentions, sequences, out_dir, min_len=5, window=4, indices=None, name="All"):
    """
    根据注意力得分，找到具有高注意力的 DNA 序列区域，并保存为 FASTA 格式。
    """
    # 如果输出目录不存在，创建它
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    # 如果注意力得分是 2D 的，说明只有一个注意力得分
    if len(attentions.shape) == 2:
        print("Processing Single Attention Scores.")
        with open(os.path.join(out_dir, "{}_high_attention.fa".format(name)), "w") as fa:
            # 对每个注意力得分和序列进行迭代
            for index, attention, sequence in zip(range(attentions.shape[0]), attentions, sequences):
                # 找到高注意力区域
                motif_regions = find_high_attention(attention, min_len)
                for motif_idx in motif_regions:
                    # 根据motif_idx转换为对应的DNA范围
                    start_dna_idx, end_dna_idx = token_range_to_dna_range(motif_idx[0], motif_idx[1], len(sequence))
                    # 考虑窗口大小进行调整
                    safe_start = max(start_dna_idx - window, 0)
                    safe_end = min(end_dna_idx + window, len(sequence))
                    if(safe_end<0):print("+++++++++++++++++++++++++++++++++++++++++++",motif_idx[0], motif_idx[1], len(sequence))
                    # 提取对应的DNA子序列
                    seq = sequence[safe_start:safe_end]
                    to_write = ">{}_{}_{}\n".format(index, safe_start, safe_end)
                    to_write += seq + "\n"
                    fa.write(to_write)

### 模型学习到的Attention矩阵

In [ ]:
### 行数等于输入序列的数量,列数等于模型的last_hidden_state中的特征数量(768)
# /home/u20111010010/Project/DNA-Pretraining/Level1/003.Sequence_Visualization/Dataset_HERV/BERT_HERV_Multi_RUN0_extract_single_attentions.npy
atten_scores=np.load(os.path.join(output_dir, "BERT_HERV_Multi_RUN0_extract_single_attentions.npy"))
#np.save(os.path.join(output_dir, "BERT_HERV_Multi_RUN0_extract_unnorm_attentions.npy"), unnorm)
#np.save(os.path.join(output_dir, "BERT_HERV_Multi_RUN0_extract_multi_attentions.npy"), multi)

### 数据和注意力得分按类型拆分

In [ ]:
### 找到 label 列中值为 1 或 2 的行的索引

indices_0 = np.where(df['multi'] == 0)[0]
indices_1 = np.where(df['multi'] == 1)[0]
indices_2 = np.where(df['multi'] == 2)[0]
indices_3 = np.where(df['multi'] == 3)[0]

In [ ]:
### 数据和注意力矩阵分割
df_0 = df.iloc[indices_0]
atten_scores_0=atten_scores[indices_0]
df_1 = df.iloc[indices_1]
atten_scores_1=atten_scores[indices_1]
df_2 = df.iloc[indices_2]
atten_scores_2=atten_scores[indices_2]
df_3 = df.iloc[indices_3]
atten_scores_3=atten_scores[indices_3]

In [ ]:
import seaborn as sns
#{"0":"Non-HERV_Coding","1":"HERV_Coding","2":"Non-HERV_Non-Coding","3":"HERV_Non-Coding"}
all_datasets = [df_0, df_1, df_2,df_3]
all_attens = [atten_scores_0,atten_scores_1,atten_scores_2,atten_scores_3]

colors = sns.color_palette("deep", 4) 
# 定义标签到颜色的映射
id2color = {
        'Non-HERV_Coding': colors[0],
        'HERV_Coding': colors[1],
        'HERV_Non-Coding': colors[2],
        'Non-HERV_Non-Coding': colors[3]
    }
all_names = ["Non-HERV_Coding", "HERV_Coding", "Non-HERV_Non-Coding","HERV_Non-Coding"]
all_colors = [id2color[all_names[0]],id2color[all_names[1]], id2color[all_names[2]], id2color[all_names[3]]]

### 平均注意力趋势（ average trend）

In [ ]:
###### 正常显示
MIN_SEQ_LEN = 8
WINDOW=4
for i in range(len(all_attens)):
    dataset = all_datasets[i]
    atten = all_attens[i]
    name = all_names[i]
    color = all_colors[i]
    '''
    # Run the Sequence Extraction and Visualization
    attention_seq(atten, dataset["seq"], min_len=MIN_SEQ_LEN, 
              window=WINDOW, name=name,
              out_dir="BERT_attention_seqs")
    print("Saved sequence file for {}.".format(name))
    '''
    ## <preprocess or split your data>
    avg = np.average(atten, 0)
    x_values = list(range(len(avg)))
    # Create a plot using matplotlib
    plt.plot(x_values, avg, linestyle="-", color=color, label=name)

    
    
plt.title(f"")  # Use 'title' instead of 'set_title'
plt.xlabel("BERT Tokens(Base)")  # Use 'xlabel' instead of 'set_xlabel'
plt.ylabel("Average Local Representation Weighted (ALRW) Scores")  # Use 'ylabel' instead of 'set_ylabel'
plt.legend(loc='upper right')
# Show the plot
#plt.subplots_adjust(left=0.1, right=0.95, bottom=0.1, top=0.95)
#plt.xticks(rotation=45, va='center', fontsize=9, fontweight='bold')

# 设置轴标题的样式
#ax.set_xlabel('X Axis Title', fontsize=9, fontweight='bold')
#ax.set_ylabel('Y Axis Title', fontsize=9, fontweight='bold')

plt.savefig(f"{dataset_name}-All_Attention-trend_visualization.pdf")
plt.savefig(f"{dataset_name}-All_Attention-trend_visualization.png")
plt.clf()

In [ ]:
###### 仅显示部分plt.ylim(0, 0.15) ############################################################################ 正文
MIN_SEQ_LEN = 8
WINDOW=4

plt.subplots(figsize=(4,3))
#plt.rcParams['xtick.labelsize'] = 8
#plt.rcParams['ytick.labelsize'] = 8
#plt.rcParams["font.family"] = prop.get_name()

for i in range(len(all_attens)):
    dataset = all_datasets[i]
    atten = all_attens[i]
    name = all_names[i]
    color = all_colors[i]
    ## <preprocess or split your data>
    avg = np.average(atten, 0)
    x_values = list(range(len(avg)))
    # Create a plot using matplotlib
    plt.plot(x_values, avg, linestyle="-", color=color, label=name)

plt.title(f" ")  # Use 'title' instead of 'set_title', #BERT Attention Plot
fontdict_bold = {'fontsize': 6} #, 'weight': 'bold'}

plt.xlabel("BERT Tokens(Base)",fontdict=fontdict_bold)  # Use 'xlabel' instead of 'set_xlabel'
plt.ylabel("Average Local Representation Weighted (ALRW) Scores",fontdict=fontdict_bold) #,fontsize=9,labelweight='bold')  # Use 'ylabel' instead of 'set_ylabel'

plt.tick_params(axis='both', which='major', labelsize=6) 
plt.legend(loc='upper left',bbox_to_anchor=(0.05, 1),fontsize=6)
plt.tight_layout() 
plt.ylim(0, 0.15)
# Show the plot
plt.subplots_adjust(left=0.1, right=0.95, bottom=0.1, top=0.95)

plt.savefig(f"{dataset_name}-All_Attention-trend_visualization_part.pdf")
plt.savefig(f"{dataset_name}-All_Attention-trend_visualization_part.png",dpi = 2000)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming 'all_attens', 'all_datasets', 'all_names', 'all_colors', 'dataset_name' are defined
# For this example, these variables need to be defined earlier in your code.
# If they are not defined, you will need to provide the data for these variables.

plt.subplots(figsize=(8,6))

# Iterate through each dataset and plot the attention scores
for i in range(len(all_attens)):
    dataset = all_datasets[i]
    atten = all_attens[i]
    name = all_names[i]
    color = all_colors[i]
    # Preprocess or split your data if necessary
    avg = np.average(atten, 0)
    x_values = list(range(len(avg)))
    plt.plot(x_values, avg, linestyle="-", color=color, label=name)

# Set the title of the plot
plt.title("BERT Attention Plot")

# Define font dictionary for bold labels
fontdict_bold = {'fontsize': 12, 'weight': 'bold'}

# Set x and y labels with bold font
plt.xlabel("BERT Tokens(Base)", fontdict=fontdict_bold)
plt.ylabel("Average Local Representation Weighted (ALRW) Scores", fontdict=fontdict_bold)

# Set other plot properties
plt.tick_params(axis='both', which='major', labelsize=9)
plt.legend(loc='upper left', bbox_to_anchor=(0.05, 1))
plt.tight_layout()
plt.ylim(0, 0.15)

# Adjust subplot parameters and save the plot
plt.subplots_adjust(left=0.1, right=0.95, bottom=0.1, top=0.95)
plt.savefig(f"{dataset_name}-All_Attention-trend_visualization_part.pdf")
plt.savefig(f"{dataset_name}-All_Attention-trend_visualization_part.png", dpi=2000)

### 展示多头注意力情况

In [ ]:
colors = [
    "blue",  # Blue
    "green",  # Green
    "red",  # Red
    "purple",  # Purple
    "orange",  # Orange
    "brown",  # Brown
    "pink",  # Pink
    "gray",  # Gray
    "olive",  # Olive
    "cyan",  # Cyan
    "magenta",  # Magenta
    "teal",  # Teal
]

def plot_multi_line(list_values):
    x_values = list(range(len(list_values[0])))
    fig, axs = plt.subplots(3, 4, figsize=(12, 9))
    axs = axs.flatten()
    for i, ax in enumerate(axs):
        ax.plot(x_values, list_values[i], linestyle="-", color=colors[i])
        ax.set_title(f"Plot for Head {i+1}")
        ax.set_xlabel("Base Pair")
        ax.set_ylabel("Average Attention for Head")
    plt.tight_layout()
    # Show the plot
    plt.show()


def plot_multi_line2(list_values, list_values1):
    x_values = list(range(len(list_values[0])))
    fig, axs = plt.subplots(3, 4, figsize=(15, 10), sharex=True)
    axs = axs.ravel()
    for i, ax in enumerate(axs):
        ax.plot(x_values, list_values[i], linestyle="-", color="red", label="Others" if i == 0 else "")
        ax.plot(x_values, list_values1[i], linestyle="-", color="green", label="HERV_Non-Coding" if i == 0 else "")
        ax.set_title(f"Plot for Head {i+1}")
        ax.set_xlabel("Base Pair")
        ax.set_ylabel("Average Attention for Head")
        # Set the y-axis limit based on the maximum value in both list_values and list_values1 for this subplot
        max_val = max(max(list_values[i]), max(list_values1[i]))
        ax.set_ylim([0, max_val * 1.1])  # Set upper limit to 10% higher than the max value
        ax.tick_params(axis='both', which='major', labelsize=9)
    # Create a single legend for the whole figure
    handles, labels = axs[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 1.02), ncol=2, fontsize='large')
    plt.tight_layout() #rect=[0, 0.03, 1, 0.95]


def plot_multi_line2(list_values, list_values1):
    x_values = list(range(len(list_values[0])))
    fig, axs = plt.subplots(3, 4, figsize=(15, 10), sharex=True)
    axs = axs.ravel()
    for i, ax in enumerate(axs):
        ax.plot(x_values, list_values[i], linestyle="-", color="red", label="Others" if i == 0 else "")
        ax.plot(x_values, list_values1[i], linestyle="-", color="green", label="HERV_Non-Coding" if i == 0 else "")
        ax.set_title(f"Plot for Head {i+1}")
        ax.set_xlabel("Base Pair")
        ax.set_ylabel("Average Attention for Head")
        # Set the y-axis limit based on the maximum value in both list_values and list_values1 for this subplot
        max_val = max(max(list_values[i]), max(list_values1[i]))
        ax.set_ylim([0, max_val * 1.1])  # Set upper limit to 10% higher than the max value
        ax.tick_params(axis='both', which='major', labelsize=9)
    # Create a single legend for the whole figure and place it below the subplots
    handles, labels = axs[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 1.0), ncol=2, fontsize='large')
    # Adjust the layout to make room for the legend
    # The rect argument defines the (left, bottom, right, top) rectangle in normalized figure coordinates that the whole subplots area (including labels) will fit into
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    # Show the plot
    plt.show()


In [ ]:
##### 所有类型序列平均得分
multi_scores=np.load(os.path.join(output_dir, "BERT_HERV_Multi_RUN0_extract_multi_attentions.npy"))
# Loading in attention scores from atten.npy. Please note that this currently uses a small sample of MHA scores.

multi_average = np.average(multi_scores, 0)
plot_multi_line(multi_average)
plt.savefig(f"{dataset_name}-All_Multi-Header_Attention-trend_visualization.pdf")

In [ ]:
##### HERV_Non-Coding
multi_scores_3=multi_scores[indices_3]
multi_average_3 = np.average(multi_scores_3, 0)
plot_multi_line(multi_average_3)
plt.savefig(f"{dataset_name}-All_Multi-Header_Attention-trend_visualization-HERV_Non-Coding.pdf")
plt.savefig(f"{dataset_name}-All_Multi-Header_Attention-trend_visualization-HERV_Non-Coding.png")

In [ ]:
##### Others
multi_scores_012=multi_scores[np.concatenate([indices_0, indices_1, indices_2])]
multi_average_012 = np.average(multi_scores_012, 0)
plot_multi_line2(multi_average_012,multi_average_3)
plt.savefig(f"{dataset_name}-All_Multi-Header_Attention-trend_visualization-HERV_Non-Coding#Other.pdf")
plt.savefig(f"{dataset_name}-All_Multi-Header_Attention-trend_visualization-HERV_Non-Coding#Other.png")

### 根据Tokens区间评估区域内DNA序列特性

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 对给定的DNA序列计算一系列的特征的函数
def evaluate_dataset(sequences, kmer=6):
    gc_contents = []
    kmer_frequencies = []
    shannon_entropies = []
    cpg_island_scores = []
    for sequence in sequences:
        if len(sequence) == 0:
            # Append NaN for sequences of length 0
            gc_contents.append(np.nan)
            kmer_frequencies.append(np.nan)
            shannon_entropies.append(np.nan)
            cpg_island_scores.append(np.nan)
            continue
        # Calculate GC content
        gc_content = (sequence.count('G') + sequence.count('C')) / len(sequence)
        gc_contents.append(gc_content)
        # Calculate kmer frequency
        if len(sequence) - kmer + 1 > 0:
            kmers = [sequence[i:i+kmer] for i in range(len(sequence) - kmer + 1)]
            kmer_freq = len(set(kmers)) / (len(sequence) - kmer + 1)
            kmer_frequencies.append(kmer_freq)
        else:
            kmer_frequencies.append(np.nan)
        # Calculate Shannon entropy
        prob = [float(sequence.count(c)) / len(sequence) for c in dict.fromkeys(list(sequence))]
        prob = [p for p in prob if p > 0]
        shannon_entropy = -sum(p * np.log2(p) for p in prob)
        shannon_entropies.append(shannon_entropy)
        # CpG island
        cg_count = sequence.count('CG')
        c_freq = sequence.count('C') / len(sequence)
        g_freq = sequence.count('G') / len(sequence)
        expected_cg_count = c_freq * g_freq * len(sequence)
        # Ensure we don't get a division by zero
        if expected_cg_count != 0:
            cpg_score = cg_count / expected_cg_count
        else:
            cpg_score = np.nan
        cpg_island_scores.append(cpg_score)
    return gc_contents, kmer_frequencies, shannon_entropies, cpg_island_scores


In [ ]:
# 为数据集中的每个标签评估DNA特征的函数
def evaluate_dna_characteristics(df, labels, token_ranges):
    results_dict = {}
    for label in labels:
        sequences = df[df['multi'] == label]['seq']
        dna_lengths = sequences.apply(len)
        results = []
        for index, dna_length in enumerate(dna_lengths):
            sequence = sequences.iloc[index]
            sub_results = []
            for start, end in token_ranges:
                start_dna_idx, end_dna_idx = token_range_to_dna_range(start, end, dna_length)
                subsequence = sequence[start_dna_idx:end_dna_idx]
                gc, kmer_freq, shannon,cpg = evaluate_dataset([subsequence])
                sub_results.append([gc, kmer_freq, shannon,cpg])
            results.append(sub_results)
        # 存储每个label的结果
        results_dict[label] = np.nanmean(results, axis=0)
    return results_dict

In [ ]:
### 相应指标计算

# 假设 df 包含了一个 'label' 列，其中的值为 "A", "B", "C" 或 "D"
labels = [0, 1, 2, 3]
id2label={"0":"Non-HERV_Coding","1":"HERV_Coding","2":"Non-HERV_Non-Coding","3":"HERV_Non-Coding"}
token_ranges = [(0, 50), (50, 150), (150, 250), (250, 350), (350, 450), (450, 512)]
results_dict = evaluate_dna_characteristics(df, labels, token_ranges)

In [ ]:
### 格式转化使其变成便于操作的DataFrame数据格式
dfs = []

for label in labels:
    data = np.squeeze(results_dict[label])  # 去除单一维度
    #df_label = pd.DataFrame(data, columns=["GC", "6-mer", "Shannon","CpG"])
    df_label = pd.DataFrame(data, columns=["GC_Content","6-mer_Freq","Shannon_Entropy","CpG-Island_Score"])
    df_label.columns = [f"{id2label[str(label)]}_{col}" for col in df_label.columns]
    dfs.append(df_label)

# 沿列轴连接所有的DataFrame
df_results = pd.concat(dfs, axis=1)

# 使用token_ranges作为行名
df_results['Token Ranges'] = [f"{start}-{end}" for start, end in token_ranges]
df_results.set_index('Token Ranges', inplace=True)

In [ ]:
### 折线图进行基因组各项特征展示（所有特征合并在一起）
import matplotlib.pyplot as plt
import numpy as np

# Assuming you've already defined df_results, token_ranges, and dataset_name
features = ["GC", "6-mer", "Shannon", "CpG"]
features1 = ["GC_Content","K-mer_Freq","Shannon_Entropy","CpG-Island_Score"]
labels = ["Non-HERV_Coding", "HERV_Coding", "Non-HERV_Non-Coding", "HERV_Non-Coding"]
# Define markers for each feature
feature_markers = {
    "GC": "o",       # Circle
    "6-mer": "s",    # Square
    "Shannon": "^",  # Triangle up
    "CpG": "D"       # Diamond
}
# Define colors for each label
feature_markers1 = {"GC_Content": "o","K-mer_Freq": "s","Shannon_Entropy": "^","CpG-Island_Score": "D" }
label_colors = id2color

fig, ax = plt.subplots()  # Increase figure size，figsize=(9, 9)
x = np.arange(len(token_ranges))

# Plot lines for each combination of label and feature
for i, feature in enumerate(features):
    for j, label in enumerate(labels):
        y_values = df_results[f"{label}_{feature}"]
        ax.plot(x, y_values, color=label_colors[label], marker=feature_markers[feature], 
                label=f"{label} {feature}" if i == 0 else "", 
                alpha=0.7, linewidth=1.5,markersize=6)  # Adjust opacity and line width

# Setting labels, title, and legends
ax.set_ylabel('Value')
ax.set_xlabel('BERT Tokens Region')  # Add X-axis label
ax.set_title('Comparison of DNA Characteristics across Different Labels and Token Ranges',fontsize=10)  ###,fontsize=4
ax.set_xticks(x)
ax.set_xticklabels([f"{start}-{end}" for start, end in token_ranges])

# Expand Y-axis limits slightly
ylim = ax.get_ylim()
y_padding = (ylim[1] - ylim[0]) * 0.4
ax.set_ylim(ylim[0], ylim[1] + y_padding)

# Create separate legends for colors and shapes
from matplotlib.lines import Line2D
legend_elements_colors = [Line2D([0], [0], color=c, lw=2, label=l) for l, c in label_colors.items()]
legend_elements_shapes = [Line2D([0], [0], color='black', marker=m, lw=0, label=f) for f, m in feature_markers1.items()]

# Add legends
leg1 = ax.legend(handles=legend_elements_colors, loc='upper left')
ax.add_artist(leg1)
ax.legend(handles=legend_elements_shapes, loc='upper right')

# Rotate x-axis labels
plt.xticks(rotation=45)
plt.tight_layout()

# Save figures
plt.savefig(f"{dataset_name}-All_Attention_Tokens-Region_DNA_Characteristics-trend_visualization-line.pdf")
plt.savefig(f"{dataset_name}-All_Attention_Tokens-Region_DNA_Characteristics-trend_visualization-line.png")
plt.show()

In [ ]:
### 折线图进行基因组各项特征展示（所有特征分开绘图）,######################################################################## 正文
# 创建子图，减少子图之间的间隙
features = ["Shannon_Entropy", "GC_Content", "CpG-Island_Score"]
labels = ["Non-HERV_Coding", "HERV_Coding", "Non-HERV_Non-Coding", "HERV_Non-Coding"]

import seaborn as sns
colors = sns.color_palette("deep", 4) 
# 定义标签到颜色的映射
id2color = {'Non-HERV_Coding': colors[0],'HERV_Coding': colors[1],'HERV_Non-Coding': colors[2],'Non-HERV_Non-Coding': colors[3]}

# 创建子图，减少子图之间的间隙
fig, axs = plt.subplots(3, 1, figsize=(4,3), sharex=True, gridspec_kw={'hspace': 0.005})   #figsize=(8,8)
plt.rcParams['xtick.labelsize'] = 3
plt.rcParams['ytick.labelsize'] = 3

handles_list, labels_list = [], []  # 存储所有图例的句柄和标签
x = np.arange(len(token_ranges))

for i, feature in enumerate(features):
    # 收集所有特征的值
    all_values = []
    ax = axs[i]
    for j, label in enumerate(labels):
        y_values = df_results[f"{label}_{feature}"]
        all_values.extend(y_values)
        #line_color = plt.rcParams['axes.prop_cycle'].by_key()['color'][j]
        #line, = ax.plot(x, y_values, color=id2color[j], alpha=0.7, linewidth=1.5, marker='o', markersize=6)
        line, = ax.plot(x, y_values, color=id2color[label], linestyle="-")
        if i == 0:  # 避免图例句柄和标签的重复添加
            handles_list.append(line)
            labels_list.append(label)
    ax.set_ylabel(feature,fontsize=6) #, fontweight='bold')  # 设置Y轴标签
    # 设置Y轴的刻度标签，首先要设置刻度位置
    y_ticks = np.linspace(min(all_values), max(all_values), 5)
    ax.set_yticks(y_ticks)
    # 然后设置刻度标签，并确保字体大小与X轴一致
    ax.set_yticklabels([f"{y:.2f}" for y in y_ticks], fontsize=5.6)
    #ax.set_yticklabels([f"{val:.2f}" for val in np.linspace(y_values.min(), y_values.max(), len(ax.get_xticks()))], fontsize=5.6)
    if i < len(features) - 1:  # 对于非最后一个子图，隐藏x轴标签
        plt.setp(ax.get_xticklabels(), visible=False)
        
# 设置最后一个子图的X轴标签
fontdict_bold = {'fontsize': 6} #, 'weight': 'bold'}

axs[-1].set_xlabel('BERT Tokens(Region)',fontdict = fontdict_bold)
axs[-1].set_xticks(x)
axs[-1].set_xticklabels([f"{start}-{end}" for start, end in token_ranges], rotation=0, ha="center",fontsize=5.6)  ##fontsize=9, fontweight='bold'

# 在子图1的右上角创建整体图例
# 注意：bbox_to_anchor的值可能需要根据您的具体布局进行微调
#legend = fig.legend(handles_list, labels_list, loc='upper left', bbox_to_anchor=(0.675, 0.8))
#plt.tight_layout(rect=[0, 0, 0.85, 1])  # 为图例留出空间
#plt.subplots_adjust(left=0.1, right=0.95, bottom=0.1, top=0.95)

# 打印绘图的默认字体
print(plt.rcParams['font.family'])
plt.subplots_adjust(left=0.1, right=0.95, bottom=0.1, top=0.95)
# Save figures
plt.savefig(f"{dataset_name}-All_Attention_Tokens-Region_DNA_Characteristics-trend_visualization-line.pdf")
plt.savefig(f"{dataset_name}-All_Attention_Tokens-Region_DNA_Characteristics-trend_visualization-line.png",dpi = 2000)


In [ ]:
### 柱状图进行基因组特征展示（分特征绘图）
features = ["6-mer_Freq", "Shannon_Entropy", "GC_Content", "CpG-Island_Score"]
labels = ["Non-HERV_Coding", "HERV_Coding", "Non-HERV_Non-Coding", "HERV_Non-Coding"]
x = np.arange(len(token_ranges))  # 每个token区间的标签位置
width = 0.2  # 柱状图的宽度

from matplotlib.backends.backend_pdf import PdfPages
with PdfPages(f"{dataset_name}-All_Attention_Tokens-Region_DNA_Characteristics-trend_visualization.pdf") as pdf:
    for i, feature in enumerate(features):
        fig, ax = plt.subplots(figsize=(15, 7))
        for j, label in enumerate(labels):
            data_label_feature = f"{label}_{feature}"
            # 在这里，我们不再将feature添加到图例的标签中
            ax.bar(x + j * width, df_results[data_label_feature], width, label=label,color=id2color[label])  # 已移除 `f"{label} {feature}"`
        # 添加标签，标题和图例
        ax.set_ylabel('Value')
        ax.set_title(f'Comparison of {feature} across Different Labels and Token Ranges')
        ax.set_xticks(x + width * (len(labels) - 1) / 2)
        ax.set_xticklabels([f"{start}-{end}" for start, end in token_ranges])
        ax.legend(loc='upper left')  # 图例现在只包含标签
        # 旋转x轴标签以避免重叠
        plt.xticks(rotation=45)
        plt.tight_layout()
        # 保存当前的图形到PDF文件
        pdf.savefig(fig)
    # plt.show()  # 这将仅在最后显示最终的图形；如果您希望每生成一个图形就显示，需要将此行移动到循环中
